In [15]:
#import

import bs4
import json
import time
import requests
import scipy as sc
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

driver=webdriver.Chrome()


starting to build our dataset using selenium

the contest started in the year 1956 BUT only at 1957 they started to rank with points so this is where we'll start

Scrolling through pages in both sites

In [16]:

def move_page_euroWorld(Syear):
    
    if Syear==2020:
        driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[2]/a[2]').click()

    driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[2]/a[2]').click()

In [17]:

# def move_page_eurovision(Syear):
#     driver.get("https://eurovision.tv/events")
#     time.sleep(6)
#     driver.find_element_by_xpath('//*[@id="onetrust-reject-all-handler"]').click()
#     # driver.find_element_by_xpath('/html/body/a').click()
#     driver.find_element(By.CSS_SELECTOR,f"[href*='{Syear}']").click()
#     driver.find_element_by_xpath('//*[@id="main"]/div[1]/div[3]/div[2]/a[3]').click()
#     driver.find_element_by_xpath('//*[@id="main"]/div[2]/div/div[1]/nav/div/a[2]').click()


In [18]:
#lists to crawl from eurovisionworld generalBoard
tablesize=0
Syear=1975
place=[]
country=[]
song=[]
artist=[]
year=[]
point=[]
normPoints=[]
normelized=[]
running=[]
contestLoc=[]
# from eurovision official site:

one=[]
two=[]
three=[]
four=[]
five=[]
six=[]
seven=[]
eight=[]
ten=[]
twelve=[]
points_year=[]

#from wikipedia
vote_array=[]
vote_dict={}
vote_dict_by_year={}

in this part we will crawl from eurovisionworld in order to recive the following: place, country,song,artist,year,point,running


In [ ]:
driver.get("https://eurovisionworld.com/eurovision/1975")
while Syear<=1976:
    location=driver.find_element_by_xpath('//*[@id="page"]/div[2]/main/div/div[1]/div[1]/span/span/a').text.split(' ')[0]
    #taking data from the general board
    general_Table =driver.find_element_by_xpath('//*[@id="voting_table"]/table/tbody')
    #from the table on EurovisionWorld 
    for row in general_Table.find_elements_by_css_selector('tr'):
        place.append(row.find_elements_by_tag_name('td')[0].text)
        country.append(row.find_elements_by_tag_name('td')[1].text)
        #spliting the song and artist from the same cell:
        song.append(row.find_elements_by_tag_name('td')[2].text.split('\n')[0])
        artist.append(row.find_elements_by_tag_name('td')[2].text.split('\n')[1])
        point.append(row.find_elements_by_tag_name('td')[3].text)
        if Syear>=2016:
            running.append(row.find_elements_by_tag_name('td')[6].text)
        else:
            running.append(row.find_elements_by_tag_name('td')[4].text)
        year.append(Syear)
        contestLoc.append(location)
    
    scoring_Table=driver.find_element_by_xpath('//*[@id="scoreboard"]/div/div[1]/table/tbody')
    table_head=driver.find_element_by_xpath('//*[@id="scoreboard"]/div/div[1]/table/thead/tr')
    to_countries=[]

    i=1
    tablesize=len(year)-tablesize
    for country_name in table_head.find_elements_by_css_selector('td'):
        if i>tablesize:
            break
        for row in scoring_Table.find_elements_by_css_selector('tr'):
            vote_array.append(row.find_elements_by_tag_name('td')[i+3].text)
            # print(i)
        vote_dict[country_name.find_element_by_xpath(f'//*[@id="scoreboard"]/div/div[1]/table/thead/tr/td[{i+1}]').get_attribute('data-from')]=vote_array
        vote_array=[]
        i+=1
    #
    for row in scoring_Table.find_elements_by_css_selector('tr'):
         to_countries.append(row.find_elements_by_css_selector('td')[2].text)
        
    
    vote_dict_by_year[Syear]=[to_countries,vote_dict]
    to_countries=[]
    country_name=[]
    vote_dict={}
    Syear+=1
    move_page_euroWorld(Syear)
    if Syear==2020:
        Syear+=1

In [20]:
pd2=pd.DataFrame(vote_dict)
pd2.index=to_countries
pd2=pd2.T
pd2

""


In [152]:
df3=pd.DataFrame(vote_dict_by_year[1975][1])
df3.index=vote_dict_by_year[1975][0]
for col in df3:
    df3[col]= pd.to_numeric(df3[col], downcast="float")
df3=df3.T
df3


,Netherlands,United Kingdom,Italy,France,Luxembourg,Switzerland,Finland,Sweden,Ireland,Spain,Israel,Malta,Yugoslavia,Monaco,Belgium,Portugal,Germany,Norway,Turkey
be,3.0,10.0,6.0,2.0,NaN,8.0,NaN,NaN,12.0,4.0,1.0,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
fi,10.0,7.0,12.0,NaN,5.0,4.0,NaN,6.0,1.0,8.0,3.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
fr,5.0,12.0,4.0,NaN,3.0,10.0,NaN,7.0,6.0,NaN,1.0,8.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
de,8.0,10.0,4.0,NaN,NaN,6.0,12.0,NaN,NaN,5.0,1.0,NaN,2.0,3.0,7.0,NaN,NaN,NaN,NaN
ie,8.0,3.0,6.0,12.0,10.0,2.0,5.0,NaN,NaN,7.0,1.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
il,12.0,10.0,5.0,7.0,6.0,NaN,8.0,3.0,NaN,4.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN
it,1.0,3.0,NaN,8.0,10.0,12.0,NaN,NaN,4.0,6.0,2.0,NaN,NaN,5.0,NaN,NaN,NaN,7.0,NaN
lu,10.0,12.0,3.0,NaN,NaN,2.0,6.0,7.0,NaN,NaN,1.0,5.0,NaN,4.0,NaN,NaN,8.0,NaN,NaN
mt,12.0,8.0,10.0,7.0,5.0,6.0,NaN,2.0,4.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN
mc,10.0,12.0,1.0,7.0,NaN,5.0,8.0,6.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0


In [173]:
test_dict={} 
# for row in df3.index:
#     for col in df3.columns:
for points in [1,2,3,4,5,6,7,8,10,12]:
    test_dict[points]=df3.index(df3['be']==points)
print(test_dict)


KeyError: 'be'

In [115]:
dict={'Country':country,'Song':song,'Artist':artist,'Year':year,'Points':point,'Running_Order':running,'Place':place,'Contest_Location':contestLoc }
df=pd.DataFrame(dict)
df["Points"] = pd.to_numeric(df["Points"], downcast="float")
# df.to_csv('eurovision_dataset.csv')
df

,Country,Song,Artist,Year,Points,Running_Order,Place,Contest_Location
0,Netherlands,Ding-A-Dong,Teach-In,1975,152.0,1,1,Stockholm
1,United Kingdom,Let Me Be The One,The Shadows,1975,138.0,9,2,Stockholm
2,Italy,Era,Wess & Dori Ghezzi,1975,115.0,19,3,Stockholm
3,France,Et Bonjour à Toi L'artiste,Nicole Rieu,1975,91.0,3,4,Stockholm
4,Luxembourg,Toi,Géraldine,1975,84.0,5,5,Stockholm
5,Switzerland,Mikado,Simone Drexel,1975,77.0,7,6,Stockholm
6,Finland,Old Man Fiddle,Pihasoittajat,1975,74.0,15,7,Stockholm
7,Sweden,"Jennie, Jennie",Lars Berghagen and the Dolls,1975,72.0,18,8,Stockholm
8,Ireland,That's What Friends Are For,The Swarbriggs,1975,68.0,2,9,Stockholm
9,Spain,Tú Volverás,Sergio & Estíbaliz,1975,53.0,17,10,Stockholm


Crawling throgh Eurovision site to find out the scoring
